Faz com que o Python mude o diretório de trabalho atual (current working directory, cwd) para a pasta que você especificou.

In [38]:
import os
os.chdir(r"C:\Users\Laura\Desktop\saeb_microdados_dashboard")

Carrega os dados de diretores de 1995 e aplica mapeamentos para tornar códigos e categorias legíveis, preparando o DataFrame para análise exploratória.

In [39]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from src.ingestion.read_diretor import read_diretor
from src.dictionaries.diretor_maps import (
    map_categorias_basicas,
    map_geografia,
    map_formacao_diretor,
    map_avaliacoes,
)

df_diretor = read_diretor(1995)
pd.set_option("display.max_rows", None)

df_diretor = map_categorias_basicas(df_diretor)
df_diretor = map_geografia(df_diretor)
df_diretor = map_formacao_diretor(df_diretor)
df_diretor = map_avaliacoes(df_diretor)



## Mapa de calor de participacao dos pais em reuniões por estado.
### 1. Preparação dos dados

In [ ]:
df = df_diretor[["uf", "participacao_pais"]].copy()

### Converte para numérico e trata valores inválidos

In [ ]:
df["participacao_pais"] = pd.to_numeric(df["participacao_pais"], errors="coerce")

### Remove registros sem informação

In [ ]:
df = df.dropna(subset=["participacao_pais"])

### Calcula a **MEDIANA** por estado (UF)

In [ ]:
df_uf = (
    df.groupby("uf", as_index=False)["participacao_pais"]
    .median()
)

### 2. Carregamento do mapa do Brasil.

In [ ]:
mapa_br = gpd.read_file("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson")

### Realiza o merge entre mapa e os dados agregados

In [ ]:
mapa_plot = mapa_br.merge(
    df_uf,
    left_on="name",
    right_on="uf",
    how="left"
)

### 3. Plotagem do mapa (mediana)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

mapa_plot.plot(
    column="participacao_pais",
    ax=ax,
    cmap="RdYlGn", # vermelho → amarelo → verde
    legend=True,
    missing_kwds={
        "color": "lightgrey",
        "label": "Sem dados"
    },
    legend_kwds={
        "orientation": "horizontal",
        "shrink": 0.7,
        "pad": 0.04,
        "label":{
            "Participação dos pais em reuniões escolares\n"
            "0.9 ≈ 90% (quase todos)\n"
            "0.8 ≈ 80%\n"
            "0.5 ≈ 50% (cerca da metade)\n"
            "0.4 ≈ 40%\n"
            "0.1 ≈ 10% (muito poucos)"
        }
    }
)

### 4. Anotação dos valores no mapa

In [ ]:
for _, row in mapa_plot.iterrows():
    if pd.notna(row["participacao_pais"]):
        ax.annotate(
            text=f"{row['participacao_pais']:.1f}",
            xy=(row.geometry.centroid.x, row.geometry.centroid.y),
            ha="center",
            va="center",
            fontsize=8,
            color="black"
        )

### 5. Ajustes estéticos e visualização

In [ ]:
ax.set_title(
    "Participação dos pais no processo de aprendizagem\n"
    "(mediana por estado — SAEB 1995)",
    fontsize=11
)

ax.axis("off")
plt.tight_layout()
plt.show()

Checar outliers em idade dos diretores.

In [ ]:
plt.figure(figsize=(6, 4))
plt.boxplot(
    df_diretor["idade"].dropna(),
    vert=False
)
plt.title("Distribuição da idade dos diretores")
plt.xlabel("Idade")
plt.show()

Checar outliers em salários de diretores (super salários).

In [ ]:
plt.figure(figsize=(6, 4))
plt.boxplot(
    df_diretor["valor_ultimo_salario"].dropna(),
    vert=False
)
plt.title("Distribuição de salário dos diretores")
plt.xlabel("salário")
plt.show()